<a href="https://colab.research.google.com/github/AliEbadi110/Natural-Language-Processing-Text-Classification-Sample-Projects/blob/main/07_NLP_Transformers_Sentence_Similarity_Custom_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP - Transformers - Sentence Similarity - Custom Dataset**

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install evaluate

In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

## 1. Loading Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Datasets/Hackathon/Problem 4/train.csv')
df.head()

,SENTENCE A,SENTENCE B,label
0,someone is dirtying an animal,a woman is cleaning a shrimp,0
1,a woman in a black dress is pulling a cart and...,a lady is dressed in black and is carrying a w...,0
2,a person is cutting garlic into pieces with a ...,someone is putting ingredients into a wok,0
3,a woman is rock climbing pausing and calculati...,a man is rock climbing and a city and a bay ar...,0
4,a rabbit is playing with a toy rabbit,there is no bunny playing with a stuffed bunny,1


In [ ]:
df.shape

(6694, 3)

In [ ]:
df_train, df_val = train_test_split(df, test_size=0.2)

In [ ]:
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "val": Dataset.from_pandas(df_val)
    })

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['SENTENCE A', 'SENTENCE B', 'label', '__index_level_0__'],
        num_rows: 5355
    })
    val: Dataset({
        features: ['SENTENCE A', 'SENTENCE B', 'label', '__index_level_0__'],
        num_rows: 1339
    })
})

In [ ]:
dataset['train'][0]

{'SENTENCE A': 'the young kids are posing with a green soccer ball in a park',
 'SENTENCE B': 'four boys are kneeling next to each other in front of a ball',
 'label': 0,
 '__index_level_0__': 1146}

## 2. Preprocessing

In [ ]:
model_name = 'bert-base-uncased'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenizer(dataset['train'][0]['SENTENCE A'], dataset['train'][0]['SENTENCE B'], truncation=True)

{'input_ids': [101, 1996, 2402, 4268, 2024, 20540, 2007, 1037, 2665, 4715, 3608, 1999, 1037, 2380, 102, 2176, 3337, 2024, 16916, 2279, 2000, 2169, 2060, 1999, 2392, 1997, 1037, 3608, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def tokenize_func(example):
  return tokenizer(example['SENTENCE A'], example['SENTENCE B'], truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_func, batched=True)
tokenized_datasets

Map:   0%|          | 0/5355 [00:00<?, ? examples/s]

Map:   0%|          | 0/1339 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['SENTENCE A', 'SENTENCE B', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5355
    })
    val: Dataset({
        features: ['SENTENCE A', 'SENTENCE B', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1339
    })
})

## 3. Train and Evaluate Model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric1 = load_metric('accuracy')
metric2 = load_metric('f1')

def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
  f1 = metric2.compute(predictions=predictions, references=labels)["f1"]

  return {"accuracy": accuracy, "f1": f1}

<ipython-input-16-8a7b90626a6b>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric1 = load_metric('accuracy')


In [ ]:
training_args = TrainingArguments('my_trainer_dir',
                                  per_device_train_batch_size=32,
                                  evaluation_strategy='epoch',
                                  save_strategy='epoch',
                                  num_train_epochs=5,
                                  load_best_model_at_end=True,
                                  )

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.210309,0.927558,0.847724
2,No log,0.196807,0.932786,0.857143
3,0.186500,0.249225,0.934279,0.869822
4,0.186500,0.276068,0.933532,0.869693
5,0.186500,0.348070,0.926811,0.859195


TrainOutput(global_step=840, training_loss=0.13225094250270297, metrics={'train_runtime': 277.4343, 'train_samples_per_second': 96.509, 'train_steps_per_second': 3.028, 'total_flos': 632211671595000.0, 'train_loss': 0.13225094250270297, 'epoch': 5.0})

## 5. Evaluate

In [ ]:
trainer.evaluate()

{'eval_loss': 0.19680722057819366,
 'eval_accuracy': 0.9327856609410008,
 'eval_f1': 0.8571428571428571,
 'eval_runtime': 3.3711,
 'eval_samples_per_second': 397.197,
 'eval_steps_per_second': 49.835,
 'epoch': 5.0}

In [ ]:
val_predictions = trainer.predict(tokenized_datasets['val'])

In [ ]:
val_preds = torch.nn.functional.softmax(torch.Tensor(val_predictions.predictions), dim=-1)

In [ ]:
val_preds = torch.argmax(val_preds, axis=-1)

In [ ]:
print(classification_report(df_val['label'], val_preds.numpy()))

              precision    recall  f1-score   support

           0       0.93      0.98      0.96       998
           1       0.93      0.79      0.86       341

    accuracy                           0.93      1339
   macro avg       0.93      0.89      0.91      1339
weighted avg       0.93      0.93      0.93      1339



In [ ]:
print(confusion_matrix(df_val['label'], val_preds.numpy()))

[[979  19]
 [ 71 270]]


## 5. Predict

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Datasets/Hackathon/Problem 4/test.csv')
df_test.head()

,SENTENCE A,SENTENCE B
0,a woman is peeling a potato,a woman is not peeling a potato
1,two boys on a couch are reading a book,two boys on a couch are playing video games
2,the man on stage isnt singing into the microphone,a man in a suit is standing at a microphone an...
3,tom is still in a deep coma,tom is still in a light coma
4,there is no dog turning on the grass and pursu...,a dog is turning on the grass and pursuing a f...


In [ ]:
dataset_test = DatasetDict({
    "test": Dataset.from_pandas(df_test)
    })
dataset_test

In [ ]:
tokenized_datasets = dataset_test.map(tokenize_func, batched=True)
tokenized_datasets

Map:   0%|          | 0/744 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['SENTENCE A', 'SENTENCE B', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 744
    })
})

In [ ]:
predictions = trainer.predict(tokenized_datasets['test'])

In [ ]:
preds = torch.nn.functional.softmax(torch.Tensor(predictions.predictions), dim=-1)

In [ ]:
preds = torch.argmax(preds, axis=-1)

In [ ]:
df = pd.DataFrame(preds.numpy(), columns=['label'])

In [ ]:
df.to_csv("output.csv")